<a href="https://colab.research.google.com/github/hongseoi/colab/blob/main/teddy_making_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##tensorflow 2.0 Dataset, batch, window, flat_map을 활용한 loader 만들기

https://teddylee777.github.io/tensorflow/dataset-batch-window

tf.data.Dataset을 활용해 다양한 데이터셋loader를 만들고 다양한 종류의 데이터셋을 모델에 feed하자

<br>

출처: 테디노트

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# dimension 1만큼 늘리기: expand_dims()

x = np.arange(20)
tf.expand_dims(x,1).shape

TensorShape([20, 1])

In [3]:
# tf의 expand_dims는 numpy의 expand_dims와 동일

np.expand_dims(x,1).shape

(20, 1)

In [5]:
# from_tensor_slices(): numpy array, list를 tensor dataset으로 변환

ds = tf.data.Dataset.from_tensor_slices([1,2,3,4,5])

for d in ds:
  print(d)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


In [6]:
ds = tf.data.Dataset.from_tensor_slices(np.arange(10))

for d in ds:
  print(d)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [10]:
# batch: 모델 학습시 batch_size 지정해 size만큼만 데이터 읽어들임
## 특히 이미지와 같이 큰 사이즈는 메모리에 한번에 올라가지 못하므로 batch를 나누어서 학습

ds = tf.data.Dataset.range(8)
ds = ds.batch(3, drop_remainder=True) # 마지막 남은 데이터를 drop할 것인지
list(ds.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5])]

In [11]:
ds = tf.data.Dataset.range(8)
for d in ds.batch(3, drop_remainder=True):
  print(d)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)


In [12]:
# window(): time series 데이터셋 생성에 유용
## window: 그룹화할 윈도우 크기(갯수)
## shift: 1 iteration당 몇개씩 이동할 것인지

ds= tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=False)

for d in ds:
  print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]
[6, 7, 8, 9]
[7, 8, 9]
[8, 9]
[9]


In [13]:
# drop_iteration True인 경우

ds= tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)

for d in ds:
  print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [14]:
# shift 2로 설정한경우

ds= tf.data.Dataset.range(10)
ds = ds.window(5, shift=2, drop_remainder=False)

for d in ds:
  print(list(d.as_numpy_iterator()))

[0, 1, 2, 3, 4]
[2, 3, 4, 5, 6]
[4, 5, 6, 7, 8]
[6, 7, 8, 9]
[8, 9]


In [15]:
# flat_map: 데이터셋에 함수 apply하고 결과를 flatten

ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w:w.batch(3))
for d in ds:
  print(d)

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
tf.Tensor([3 4], shape=(2,), dtype=int64)
tf.Tensor([1 2 3], shape=(3,), dtype=int64)
tf.Tensor([4 5], shape=(2,), dtype=int64)
tf.Tensor([2 3 4], shape=(3,), dtype=int64)
tf.Tensor([5 6], shape=(2,), dtype=int64)
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
tf.Tensor([6 7], shape=(2,), dtype=int64)
tf.Tensor([4 5 6], shape=(3,), dtype=int64)
tf.Tensor([7 8], shape=(2,), dtype=int64)
tf.Tensor([5 6 7], shape=(3,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [16]:
ds = tf.data.Dataset.range(10)
ds = ds.window(5, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w:w.batch(5))

for d in ds:
  print(d)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int64)
tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int64)
tf.Tensor([3 4 5 6 7], shape=(5,), dtype=int64)
tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int64)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int64)


In [19]:
# shuffle: dataset 섞음. 학습전필수!!

ds = tf.data.Dataset.from_tensor_slices(np.arange(10)).shuffle(buffer_size=5) #버퍼사이즈는 데이터셋 전체크기보다 크거나 같아야함
for d in ds:
  print(d)

tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)


In [21]:
# map: pandas의 map과 유사
## time series dataset을 만드는 경우 train/label값 분류용으로 활용 가능

window_size = 5

ds = tf.data.Dataset.range(10)
ds = ds.window(window_size, shift=1, drop_remainder=True)
ds = ds.flat_map(lambda w: w.batch(window_size))
ds = ds.shuffle(10)

ds = ds.map(lambda x:(x[:-1], x[-1:]))
for x,y in ds:
  print("train data:",x)
  print("label set:",y)

train data: tf.Tensor([4 5 6 7], shape=(4,), dtype=int64)
label set: tf.Tensor([8], shape=(1,), dtype=int64)
train data: tf.Tensor([1 2 3 4], shape=(4,), dtype=int64)
label set: tf.Tensor([5], shape=(1,), dtype=int64)
train data: tf.Tensor([5 6 7 8], shape=(4,), dtype=int64)
label set: tf.Tensor([9], shape=(1,), dtype=int64)
train data: tf.Tensor([3 4 5 6], shape=(4,), dtype=int64)
label set: tf.Tensor([7], shape=(1,), dtype=int64)
train data: tf.Tensor([0 1 2 3], shape=(4,), dtype=int64)
label set: tf.Tensor([4], shape=(1,), dtype=int64)
train data: tf.Tensor([2 3 4 5], shape=(4,), dtype=int64)
label set: tf.Tensor([6], shape=(1,), dtype=int64)
